# Airline Incidents Safety Data Jupyter Notebook

Data is from https://www.kaggle.com/datasets/tarique7/airline-incidents-safety-data

Last Update: Mar 18 2023

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/Airline Occurences.csv')

In [3]:
data.head()

,Report,Part Failure,Occurence Nature condition,Occurence Precautionary Procedures
0,MECHANICAL / LANDING GEAR GND FAIL MSG AFTER T...,RT MLG BRAKE DAMAGED,WARNING INDICATION,OTHER
1,THE NOSE LANDING GEAR DID NOT EXTEND FULLY DUR...,ZONE 700 MALFUNCTIONED,WARNING INDICATION,ABORTED APPROACH
2,THE LEFT SIDE HYDRAULIC SYSTEM FILTER BOWL ASS...,HYD FILTER FAILED,OTHER,ABORTED APPROACH
3,AIRCRAFT WAS ON ROLLOUT DURING A NORMAL LANDIN...,LEFT COLLAPSED,OTHER,OTHER
4,UPON TAKEOFF ROLL BUT PRIOR TO REACHING 80 KNO...,ZONE 600 CRACKED,WARNING INDICATION,ABORTED TAKEOFF


In [4]:
data['Occurence Nature condition'].value_counts()

OTHER                         82172
WARNING INDICATION             7154
NO TEST                        3577
SMOKE/FUMES/ODORS/SPARKS       2166
FALSE WARNING                  1683
FLUID LOSS                      975
FLT CONT AFFECTED               320
OVER TEMP                       316
NO WARNING INDICATION           255
VIBRATION/BUFFET                228
INADEQUATE Q C                  223
AFFECT SYSTEMS                  202
F.O.D.                          191
PARTIAL RPM/PWR LOSS            152
MULTIPLE FAILURE                107
FLAME/FIRE                       65
ENGINE STOPPAGE                  56
SIGNIFICANT FAILURE REPORT       53
INFLIGHT SEPARATION              47
ENGINE FLAMEOUT                  38
ELECT. POWER LOSS-50 PC          27
FLT. ATTITUDE INST.              21
Name: Occurence Nature condition, dtype: int64